# Влияние ошибок округления на решение СЛАУ. Числа обусловленности.

In [44]:
import numpy as np
import sys
from scipy.linalg import hilbert

### В качестве тестовых данных используются:                                                                                                       

In [79]:
# 1. Произвольная матрица пятого порядка
randomMatrix = np.random.random((5, 5))
print('\n'.join(' '.join(str(cell) for cell in row) for row in randomMatrix))
print()

# 2. Матрица Гильберта третьего порядка
matrixHilbert3 = np.array(hilbert(3))
print('\n'.join(' '.join(str(cell) for cell in row) for row in matrixHilbert3))
print()

# 3. Матрица Гильберта четвертого порядка
matrixHilbert4 = np.array(hilbert(4))
print('\n'.join(' '.join(str(cell) for cell in row) for row in matrixHilbert4))
print()

# 4 и 5 — матрицы из методички Пакулиной

0.9120611047026249 0.7014786677673985 0.3048611615011695 0.47414821075633296 0.9610547171506026
0.38955873288305654 0.8963349827307479 0.36784645816869577 0.13949750821727036 0.6384855659481132
0.4774538825351208 0.15127674972841665 0.966562142798751 0.3400203533363636 0.108709037909965
0.5558380185431117 0.173060241721939 0.016704992070836422 0.6821489638687249 0.4681118877644639
0.8051626097600515 0.0794638221927958 0.1509389343690658 0.3221351607168218 0.794514781532708

1.0 0.5 0.3333333333333333
0.5 0.3333333333333333 0.25
0.3333333333333333 0.25 0.2

1.0 0.5 0.3333333333333333 0.25
0.5 0.3333333333333333 0.25 0.2
0.3333333333333333 0.25 0.2 0.16666666666666666
0.25 0.2 0.16666666666666666 0.14285714285714285



### Нахождение b

In [10]:
def findB(A, xAcc=np.array([None])):
    if xAcc[0] is None:
        xAcc = np.random.uniform(-100, 100, size=A.shape[0])
    b = np.dot(A, xAcc)
    print(b)
    return b, xAcc

### Нахождение чисел обусловленности

In [11]:
def cond(A):
    v0 = 1
    mult = np.zeros(A.shape[0])
    
    # Спектральный критерий
    condSpectral = np.dot(np.linalg.norm(A), np.linalg.norm(np.linalg.inv(A)))
    for i in range(A.shape[0]):
        v0 *= np.linalg.norm(A[i])

    # Объемный критерий
    condVolumetric = v0 / abs(np.linalg.det(A))
    C = np.linalg.inv(A)
    for i in range(A.shape[0]):
        mult[i] = np.linalg.norm(A[i]) * np.linalg.norm(C[:, i])
    
    # Угловой критерий
    condAngle = max(mult)
    return condSpectral, condVolumetric, condAngle

### Варьирование матриц

In [12]:
def varied_matrix(A, b):
    k = 0
    xInacc = np.zeros((3, A.shape[0]))
    for i in (-2, -5, -8):
        AInacc = A - 10 ** i
        bInacc = b - 10 ** i
        for j in range(A.shape[0]):
            xInacc[k] = np.linalg.solve(A, bInacc)
        k += 1
    return xInacc

In [ ]:
random_matrix = numpy.random.random((num_rows, num_columns))

### Вывод данных

In [13]:
def print_matrix(A, xAcc, x_inacc):
    print("Матрица:")
    print(*A, sep='\n')
    print()
    print("Точное решение:")
    for elem in xAcc:
        print('|{:12.8f}'.format(elem), '|')
    print()
    print("Числа обусловленности:")
    print("    Спектральный критерий обусловленности:", condSpectral)
    print("    Объемный критерий (критерий Ортеги):", condVolumetric)
    print("    Угловой критерий:", condAngle)
    print()
    k = 0
    for i in (-2, -5, -8):
        print("eps = 10^({}):".format(i))
        print("    Приближенное решение:")
        for elem in x_inacc[k]:
            print("   ", '|{:16.8f}'.format(elem), '|')
        print("    Невязка:")
        for elem in abs(xAcc - x_inacc[k]):
            print("   ", '|{:16.12f}'.format(elem), '|')
        k += 1
    print()

In [84]:
data = []
with open("tests.txt") as f:
    for line in f:
        if line == '\n':
            A = np.array(data, dtype=float)
            b, xAcc = find_b(A)
            condSpectral, condVolumetric, condAngle = cond(A)
            x_inacc = varied_matrix(A, b)
            print_matrix(A, xAcc, x_inacc)
            print("{:-^100}".format(""))
            data.clear()
            continue
        else:
            data.append([float(x) for x in line.split()])


[-155.29768802 -138.30807563 -101.90784242  -23.11908475 -102.62927743]
Матрица:
[0.9120611  0.70147867 0.30486116 0.47414821 0.96105472]
[0.38955873 0.89633498 0.36784646 0.13949751 0.63848557]
[0.47745388 0.15127675 0.96656214 0.34002035 0.10870904]
[0.55583802 0.17306024 0.01670499 0.68214896 0.46811189]
[0.80516261 0.07946382 0.15093893 0.32213516 0.79451478]

Точное решение:
|-88.67908029 |
|-57.60381757 |
|-79.21477203 |
| 93.67628261 |
|-56.47548708 |

Числа обусловленности:
    Спектральный критерий обусловленности: 4.162153158416973
    Объемный критерий (критерий Ортеги): 1.250311292614698
    Угловой критерий: 1.2503112926146984

eps = 10^(-2):
    Приближенное решение:
    |    -88.62817179 |
    |    -57.58693228 |
    |    -79.23848268 |
    |     93.65170887 |
    |    -56.52688507 |
    Невязка:
    |  0.050908494013 |
    |  0.016885289614 |
    |  0.023710643954 |
    |  0.024573745178 |
    |  0.051397985053 |
eps = 10^(-5):
    Приближенное решение:
    |    -88.679